In [60]:
import pandas as pd

data = pd.read_csv('../csv/cleaned/cleaned_variations_review_posts2.csv')
data

,tourist_destination,review_post,removed_special_charas_review_post,to_lower_case_review_post,removed_stop_words_review_post,lemmatized_words_review_post
0,Burnham Park,An excellent place for family and friends to v...,An excellent place for family and friends to v...,an excellent place for family and friends to v...,excellent place family friends visit entrance ...,excellent place family friend visit entrance f...
1,Burnham Park,A lovely place to be on a nice day.We were ple...,A lovely place to be on a nice day We were ple...,a lovely place to be on a nice day we were ple...,lovely place nice day pleasantly surprised ple...,lovely place nice day pleasantly surprised ple...
2,Burnham Park,Definitely one of the best places to visit in ...,Definitely one of the best places to visit in ...,definitely one of the best places to visit in ...,definitely best places visit baguio burnham pa...,definitely best place visit baguio burnham par...
3,Burnham Park,I have always been looking at this park ever s...,I have always been looking at this park ever s...,i have always been looking at this park ever s...,looking park chance review park flexed view pa...,looking park chance review park flexed view pa...
4,Burnham Park,Very beautiful park with boating and cycling a...,Very beautiful park with boating and cycling a...,very beautiful park with boating and cycling a...,beautiful park boating cycling available insid...,beautiful park boating cycling available insid...
...,...,...,...,...,...,...
14154,Teacher's Camp,There is no ghost 👻👻👻Very affordable,There is no ghost Very affordable,there is no ghost very affordable,ghost affordable,ghost affordable
14155,Teacher's Camp,Classy place,Classy place,classy place,classy place,classy place
14156,Teacher's Camp,A beautiful view is cheap,A beautiful view is cheap,a beautiful view is cheap,beautiful view cheap,beautiful view cheap
14157,Teacher's Camp,well done,well done,well done,NaN,NaN


In [73]:
tourist_destinations = data['tourist_destination']
print(len(tourist_destinations.unique()))

19


In [74]:
tourist_destinations

0          Burnham Park
1          Burnham Park
2          Burnham Park
3          Burnham Park
4          Burnham Park
              ...      
14154    Teacher's Camp
14155    Teacher's Camp
14156    Teacher's Camp
14157    Teacher's Camp
14158    Teacher's Camp
Name: tourist_destination, Length: 14159, dtype: object

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer



tfv = TfidfVectorizer(ngram_range = (1, 1))
tfv_text = tfv.fit_transform(data['lemmatized_words_review_post'].values.astype('U'))

words = tfv.get_feature_names_out()

In [63]:
tfv_text

<14159x10341 sparse matrix of type '<class 'numpy.float64'>'
	with 140804 stored elements in Compressed Sparse Row format>

In [64]:
import gensim

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

words = list(sent_to_words(data['lemmatized_words_review_post'].values.tolist()))
print(words[0])

['excellent', 'place', 'family', 'friend', 'visit', 'entrance', 'fee', 'lake', 'ride', 'boat', 'park', 'ground', 'ride', 'small', 'bike', 'place', 'recommend', 'night', 'get', 'quite', 'dark', 'spot', 'morning', 'superb', 'experience', 'definitely', 'want', 'come']


In [65]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(words)
# Create Corpus
texts = words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]


In [66]:
dict_ = corpora.Dictionary(words)
print(dict_)

Dictionary(10303 unique tokens: ['bike', 'boat', 'come', 'dark', 'definitely']...)


In [67]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
doc_term_matrix = [dict_.doc2bow(i) for i in words]
print(doc_term_matrix[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]


In [68]:
# Build LDA model
from gensim.models import CoherenceModel

lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=19, passes=20, 
random_state=20, chunksize = 10000, eval_every = 10, iterations = 100)



In [69]:
from pprint import pprint

# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.032*"awesome" + 0.015*"like" + 0.013*"place" + 0.012*"view" + 0.011*"lot" '
  '+ 0.010*"people" + 0.009*"local" + 0.009*"good" + 0.008*"display" + '
  '0.008*"season"'),
 (1,
  '0.063*"art" + 0.035*"great" + 0.034*"museum" + 0.020*"view" + 0.017*"place" '
  '+ 0.017*"cafe" + 0.017*"work" + 0.017*"artist" + 0.016*"nice" + '
  '0.015*"artwork"'),
 (2,
  '0.025*"inside" + 0.018*"nan" + 0.018*"went" + 0.017*"building" + '
  '0.014*"allowed" + 0.012*"people" + 0.012*"hotel" + 0.010*"line" + '
  '0.009*"long" + 0.009*"slippery"'),
 (3,
  '0.054*"place" + 0.020*"nice" + 0.017*"time" + 0.012*"view" + 0.012*"photo" '
  '+ 0.012*"good" + 0.011*"flower" + 0.010*"area" + 0.009*"spend" + '
  '0.008*"enjoy"'),
 (4,
  '0.043*"horse" + 0.026*"view" + 0.023*"riding" + 0.023*"park" + 0.018*"ride" '
  '+ 0.016*"place" + 0.011*"mine" + 0.011*"mansion" + 0.010*"horseback" + '
  '0.009*"baguio"'),
 (5,
  '0.030*"view" + 0.023*"entrance" + 0.023*"place" + 0.021*"fee" + '
  '0.019*"peso" + 0.018*"pi

In [70]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.5220164859917509


In [109]:
# For Machine Based
from sklearn.model_selection import train_test_split
vectors_train, vectors_test, topics_train, topics_test = train_test_split(tfv_text, tourist_destinations, test_size = 0.1, random_state = 50)

# For Rule Based
texts_train, texts_test, topics2_train, topics2_test = train_test_split(tfv_text, tourist_destinations, test_size = 0.1, random_state = 50)

In [102]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer()
vectors = matrix.fit_transform(data['lemmatized_words_review_post'].values.astype('U')).toarray()

# For Machine Based
from sklearn.model_selection import train_test_split
vectors_train, vectors_test, topics_train, topics_test = train_test_split(vectors, tourist_destinations, test_size = 0.10, random_state = 50)

# For Rule Based
texts_train, texts_test, topics2_train, topics2_test = train_test_split(vectors, tourist_destinations, test_size = 0.10, random_state = 50)

In [110]:
from sklearn import svm
clf = svm.SVC(kernel = "linear")
clf.fit(vectors_train, topics_train)

# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()
# clf.fit(vectors_train.toarray(), topics_train)

# from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
# clf = DecisionTreeClassifier()
# clf.fit(vectors_train, topics_train)

SVC(kernel='linear')

In [115]:
from tabulate import tabulate

topics_pred_svm = clf.predict(vectors_test)

actual_predict = pd.DataFrame()
actual_predict["actual"] = topics_test
actual_predict["predict"] = topics_pred_svm

actual_predict

,actual,predict
6000,Our Lady of the Atonement Cathedral (Baguio Ca...,Our Lady of the Atonement Cathedral (Baguio Ca...
6882,Our Lady of the Atonement Cathedral (Baguio Ca...,Our Lady of the Atonement Cathedral (Baguio Ca...
4341,Wright Park,Wright Park
5646,Baguio Botanical Garden,Baguio Botanical Garden
7878,Lion’s Head,Lion’s Head
...,...,...
10113,Heritage Hill and Nature Park Garden,Heritage Hill and Nature Park Garden
366,Burnham Park,Burnham Park
4954,Wright Park,Wright Park
3473,Baguio Night Market,Baguio Night Market


In [116]:
from sklearn.metrics import classification_report
print(classification_report(topics_test, topics_pred_svm))

# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(topics_test, topics_pred_svm))

                                                                                       precision    recall  f1-score   support

                                                              Baguio Botanical Garden       0.47      0.63      0.54       109
                                                                   Baguio City Market       0.70      0.76      0.73        93
                                                Baguio Country Club Christmas Village       0.46      0.42      0.44        40
                                                                  Baguio Night Market       0.68      0.74      0.71        99
                                                                          Bell Church       0.61      0.43      0.50        47
                                                                        BenCab Museum       0.69      0.68      0.68        84
                                                                         Burnham Park       0.51      0.54    

In [114]:
import pickle

# load it again
with open('../models/svm_classifier.pkl', 'rb') as fid:
    clf = pickle.load(fid)

In [113]:
import pickle

# save the classifier
with open('../models/svm_classifier.pkl', 'wb') as fid:
    pickle.dump(clf, fid)    

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

test_df  = pd.DataFrame({'review' : ['I want to pick fresh strawberries']})

tfv2 = TfidfVectorizer(ngram_range = (1, 1), stop_words = "english")
tfv_text2 = tfv2.fit_transform([test_df['review'][0]]).todense()

words2 = tfv2.get_feature_names_out()

In [91]:
print(clf.predict(tfv_text2))

d:\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


ValueError: X has 4 features, but SVC is expecting 10341 features as input.